### Scrapping des données :

In [2]:
import bs4
import lxml
import pandas
import urllib

from urllib import request

In [3]:
from selenium import webdriver
browser = webdriver.Firefox()
browser.get('https://www.apec.fr/candidat/recherche-emploi.html/emploi?motsCles=data&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&page=0')

In [4]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

## Tests :

In [5]:
conteneur_des_resultats = browser.find_element(By.CLASS_NAME,"container-result")# recuprer la div qui contient l'ensemble d'annonces
resultats = conteneur_des_resultats.find_elements(By.XPATH,"./div/a") # Recuperer les liens vers le descriptif de chaque annonce
resultats.pop()
for element in resultats:
    print(element.get_attribute('href'))

# #CE code avec soup n'est pas correcte mais il existe un code equivalent dans lequel on recupere toutes les div (fils directe) et pout chaque dic on recuper elemnt a et son attribut href
# soup = bs4.BeautifulSoup(browser.page_source,'lxml')

# # Trouver la div qui contient l'ensemble des annonces
# conteneur_des_resultats = soup.find('div', class_='container-result')
# print(conteneur_des_resultats)
# # Si la div est trouvée, récupérer les liens des annonces
# if conteneur_des_resultats:
#     resultats = conteneur_des_resultats.find_all('div/a')[:-1]  # Récupérer tous les liens sauf le dernier

#     # Afficher les liens
#     for element in resultats:
#         lien = element.get('href')
#         if lien:
#             print(lien)
# print(len(resultats))

https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/172982802W?motsCles=data&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&page=0&selectedIndex=0
https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/172631521W?motsCles=data&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&page=0&selectedIndex=1
https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/172934112W?motsCles=data&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&page=0&selectedIndex=2
https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/172922452W?motsCles=data&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&page=0&selectedIndex=3
https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/172920679W?motsCles=data&typesConvention=143684&typesConvention=143685&typesConventio

## Changement de page :
dans cette partie on doit sauvegarder a chaque fois la page precedente afin de pouvoir y retourner ou peut etre pas ! a savoir: Les etapes a suivre sont donc les suivantes : 
-   Dans un premier lieu on genere la premiere url en  specifiant le mot cle souhaité qu'on va passer au main (ou pour cette etape on peut directemnt prendre la premiere url en dure comme etant directeemnt la la page de recherche resultatnte dans apec a la saisie de data dans la recherche)
-   Ensuite on recupere le nombre de page de resultats
-   Ensuite à get_url on va parcour toutes ces pages de resultas
-   POur chaque page on recupère la div ayant le nom de class 'container-result' qui contient la liste des dic de chaque annonce
-   On récupere pour chaque annonce le lien de sa description détaillée 
-   On se met sur la page de description on scrappe nos donnnees
-   On parcourt de cette mannière toutes les pages ainsi que toutes les offres 
-   ET voilouuuu

In [6]:
#----------NE PAS REGARDER ÇA POUR LE MOMENT LOL---------#
# premiere_page_url = browser.current_url
# # Revenir à la première page en utilisant l'historique du navigateur
# browser.get(premiere_page_url)


## Changement de page (TODO APRES)

### Ce bout de code est a utilsier pour passer a la page de description de chaque annonce :

In [7]:
# browser.execute_script('arguments[0].target="_self"; arguments[0].click();', "https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/172982802W?motsCles=data&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&page=0&selectedIndex=0")
script = 'window.location.href = "'+resultats[0].get_attribute('href')+'";'
browser.execute_script(script)

In [8]:
# script2 = 'window.location.href = "https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/172920679W?motsCles=data&typesConvention=143684&typesConvention=143685&typesConvention=143686&typesConvention=143687&page=0&selectedIndex=5";'
# browser.execute_script(script2)

In [9]:
def get_url(position, location):
        template = 'https://www.indeed.com/jobs?q={}&l={}'
        url = template.format(position, location)
        return url

## Tache 1 : scraper la page détaillée de l'annonce :
POur le moment ici comme on ne parcourt pas encore les page il faut decomenter les 2 derniere ligne afin de charger(la premiere annonce par exemple) dans le browser


In [10]:
# Ne pas executer plusieurs ici on passe sur la page detaillée de la première annonce 
# d'où le zérro ! Donc dans browser on a le contenu de la nouvelle page !
# script = 'window.location.href = "'+resultats[0].get_attribute('href')+'";'
# browser.execute_script(script)

#### Récupèration des informations du réctangle en haut de la page :
    -   Ici je récupère le nom de l'entreprise, type du contrat proposé, departement , date de publication, date d'actualisation 

In [22]:
informations_cles = ['entreprise','type_contrat','deparetement','date_publication','date_mise_a_jour','salaire','prise_de_poste','experience','metier','statut_du_poste','zone_de_deplacement','secteur_d_activite','Teletravail','descriptif_poste','profil_recherche','langues','savoir_etre','savoir_faire','descriptif_entreprise']

# Initialiser un dictionnaire avec des clés et des valeurs initiales à une liste vide
dictionnaire_annonces ={}
for cle in informations_cles:
    dictionnaire_annonces[cle]=[]
dictionnaire_annonces

{'entreprise': [],
 'type_contrat': [],
 'deparetement': [],
 'date_publication': [],
 'date_mise_a_jour': [],
 'salaire': [],
 'prise_de_poste': [],
 'experience': [],
 'metier': [],
 'statut_du_poste': [],
 'zone_de_deplacement': [],
 'secteur_d_activite': [],
 'Teletravail': [],
 'descriptif_poste': [],
 'profil_recherche': [],
 'langues': [],
 'savoir_etre': [],
 'savoir_faire': [],
 'descriptif_entreprise': []}

In [23]:
# entreprise , type_contrat , departement
entete_annonce = browser.find_element(By.CLASS_NAME,"card-offer__text")# recuprer la div du petit réctangle
informations = entete_annonce.find_elements(By.XPATH,"./ul/li") # Recuperer les liens vers le descriptif de chaque annonce
for i,info in enumerate(informations):
    cle = informations_cles[i]
    mot = info.text.strip()
    dictionnaire_annonces[cle].append(mot)



# 'date_publication','date_mise_a_jour'
date_publication = entete_annonce.find_elements(By.CLASS_NAME,"date-offre")[0].text
date_mise_a_jour = entete_annonce.find_elements(By.CLASS_NAME,"date-offre")[1].text
dictionnaire_annonces['date_publication'].append(date_publication)
dictionnaire_annonces['date_mise_a_jour'].append(date_mise_a_jour)


# 'salaire','prise_de_poste','experience','metier','statut_du_poste','zone_de_deplacement','secteur_d_activite','Teletravail'
apec_poste_imformations = browser.find_element(By.TAG_NAME,"apec-poste-informations")# recuprer la div du petit réctangle
liste_informations = apec_poste_imformations.find_elements(By.XPATH,"//div[contains(@class, 'details-post')]")
salaire = None
prise_de_poste= None
experience = None
metier = None
statut_du_poste = None
zone_de_deplacement = None
secteur_d_activite = None
Teletravail = None
for information in liste_informations:
    reponse = information.find_elements(By.XPATH, "./h4[text()='Salaire']/following-sibling::span")
    if reponse :
        salaire = reponse[0].text
        
    reponse = information.find_elements(By.XPATH, "./h4[text()='Prise de poste']/following-sibling::span")
    if reponse :
        prise_de_poste = reponse[0].text
        
    reponse = information.find_elements(By.XPATH, "./h4[text()='Expérience']/following-sibling::span")
    if reponse :
        experience = reponse[0].text
        
    reponse = information.find_elements(By.XPATH, "./h4[text()='Métier']/following-sibling::span")
    if reponse :
        metier = reponse[0].text

    reponse = information.find_elements(By.XPATH, "./h4[text()='Statut du poste']/following-sibling::span")
    if reponse :
        statut_du_poste = reponse[0].text
        
    reponse = information.find_elements(By.XPATH, "./h4[text()='Zone de déplacement']/following-sibling::span")
    if reponse :
        zone_de_deplacement = reponse[0].text

    reponse = information.find_elements(By.XPATH, "./h4[text()='Secteur d’activité du poste']/following-sibling::span")
    if reponse :
        secteur_d_activite = reponse[0].text     
    
    reponse = information.find_elements(By.XPATH, "./h4[text()='Télétravail']/following-sibling::span")
    if reponse :
        Teletravail = reponse[0].text    

dictionnaire_annonces['salaire'].append(salaire)
dictionnaire_annonces['prise_de_poste'].append(prise_de_poste)
dictionnaire_annonces['experience'].append(experience)
dictionnaire_annonces['metier'].append( metier)
dictionnaire_annonces['statut_du_poste'].append(statut_du_poste)
dictionnaire_annonces['zone_de_deplacement'].append(zone_de_deplacement)
dictionnaire_annonces['secteur_d_activite'].append(secteur_d_activite)
dictionnaire_annonces['Teletravail'].append(Teletravail)


#'descriptif_poste' 'profil_recherche' 
descriptifs = apec_poste_imformations.find_elements(By.XPATH,"//div[contains(@class, 'details-post')]")
descriptif_poste = None
descriptif_profil = None
for desc in descriptifs:
    reponse = desc.find_elements(By.XPATH, "./h4[text()='Descriptif du poste']/following-sibling::p")
    if reponse:
        descriptif_poste=reponse[0].text
    reponse = desc.find_elements(By.XPATH, "./h4[text()='Profil recherché']/following-sibling::p")
    if reponse:
        descriptif_profil=reponse[0].text

dictionnaire_annonces['descriptif_poste'].append(descriptif_poste)
dictionnaire_annonces['profil_recherche'].append(descriptif_profil)

In [42]:
langues_requises = apec_poste_imformations.find_element(By.CLASS_NAME,"added-skills-manager__language")
liste_langues_requises= langues_requises.find_elements(By.XPATH, "//h5[text()='Langues']/../following-sibling::*")
for langue in liste_langues_requises:
    print(langue.text)
    # dictionnaire_annonces['descriptif_poste'].append(descriptif_poste)



Aucune langue attendue


In [18]:
dictionnaire_annonces

{'entreprise': ['AI&Data'],
 'type_contrat': ['1 \n CDI'],
 'deparetement': ['Boulogne-Billancourt - 92'],
 'date_publication': ['Publiée le 01/02/2024'],
 'date_mise_a_jour': ['Actualisée le 01/02/2024'],
 'salaire': ['A négocier'],
 'prise_de_poste': ['Dès que possible'],
 'experience': ['Minimum 2 ans'],
 'metier': ['Data analyst'],
 'statut_du_poste': ['Cadre du secteur privé'],
 'zone_de_deplacement': ['Nationale'],
 'secteur_d_activite': ['TRAITEMENT DE DONNÉES, HÉBERGEMENT ET ACTIVITÉS CONNEXES'],
 'Teletravail': [None],
 'descriptif_poste': ["AI&Data  recherche aujourd'hui des Data Analysts expérimentés (2 ans d'expérience minimum) avec une appétence pour le Marketing et ayant de l expérience autour de l exploitation de données clients.\nNos consultants aident les entreprises à tirer le meilleur parti des données, à mettre en place des solutions opérationnelles, à identifier et mettre en œuvre les projets liés à la transformation digitale.\nIl s'agira d'intégrer notre Pôle Cust

## Toujours garder ce quit pour fermer la fenetre ouverte à la fin :

In [ ]:
browser.quit()